In [0]:
!pip install -U torch

Requirement already up-to-date: torch in /usr/local/lib/python3.6/dist-packages (1.4.0)


In [0]:
!cp "/content/drive/My Drive/all_jets_test_4D_100_percent.pkl" /content

In [0]:
!cp "/content/drive/My Drive/all_jets_train_4D_100_percent.pkl" /content

In [0]:
BIN = '/'

In [0]:

import numpy as np
from numpy import exp
from matplotlib import pyplot as plt
import math, os
import matplotlib.pyplot as plt
import pandas as pd

import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
%matplotlib inline
import sys

sys.path.append(BIN)
#from sklearn.model_selection import train_test_split

import torch.utils.data
from torch.autograd import Variable

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

#import my_matplotlib_style as ms

from fastai import data_block, basic_train, basic_data
from fastai.callbacks import ActivationStats
import fastai

import matplotlib as mpl
#mpl.rc_file(BIN + 'my_matplotlib_rcparams')
from __future__ import print_function



In [0]:
train = pd.read_pickle(BIN + 'content/all_jets_train_4D_100_percent.pkl')
test = pd.read_pickle(BIN+ 'content/all_jets_test_4D_100_percent.pkl')

In [0]:

BATCH_SIZE = 128
LOG_INTERVAL = 10
EPOCHS = 10
ZDIMS = 3

In [7]:
test

,m,pt,phi,eta
85972,4983.729980,23798.070312,1.962157,-0.059532
38742,5435.273438,21881.867188,1.035412,0.734343
128711,5239.408691,24608.134766,-1.121055,0.828848
28751,14121.240234,203110.953125,0.324205,-2.571108
131358,3344.826660,24897.294922,0.395331,1.440069
...,...,...,...,...
133326,31842.511719,236302.125000,-2.982530,0.128011
12875,5837.973145,26775.214844,-0.797112,1.042910
111439,4847.191895,25209.994141,-2.181372,-2.281394
18479,7629.757324,37038.746094,-2.670758,-0.830225


In [0]:
train_mean = train.mean()
train_std = train.std()

train = (train - train_mean) / train_std
test = (test - train_mean) / train_std

train_x = train
test_x = test
train_y = train_x  # y = x since we are building and AE
test_y = test_x



In [0]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(4, 8)
        self.fc2 = nn.Linear(8,16)
        self.fc3 = nn.Linear(16,32)
        self.fc4 = nn.Linear(32, 16)
        self.fc5 = nn.Linear(16, 8)
        self.fc61 = nn.Linear(8, 3)
        self.fc62 = nn.Linear(8,3)
        self.fc7 = nn.Linear(3, 8)
        self.fc8 = nn.Linear(8,16)
        self.fc9 = nn.Linear(16,32)
        self.fc10 =nn.Linear(32,16)
        self.fc11 =nn.Linear(16,8)
        self.fc12 =nn.Linear(8,4)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc2(h1))
        h3 = F.relu(self.fc3(h2))
        h4 = F.relu(self.fc4(h3))
        h5 = F.relu(self.fc5(h4))
        return self.fc61(h5), self.fc62(h5)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h7 = F.relu(self.fc7(z))
        h8 = F.relu(self.fc8(h7))
        h9 = F.relu(self.fc9(h8))
        h10 = F.relu(self.fc10(h9))
        h11 = F.relu(self.fc11(h10))
        return (self.fc12(h11))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

    class Loss(nn.Module):
        def __init__(self):

          super(Loss, self).__init__()
          self.mse_loss = nn.MSELoss(reduction="sum")
        def forward(self, recon_x, x, mu, logvar):
          MSE = self.mse_loss(recon_x, x)
          KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2)-logvar.exp())
          return MSE + KLD


In [0]:
model = VAE()

In [0]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

In [0]:
def df_to_tensor(df):
    device=get_device()
    return torch.from_numpy(df.values).float().to(device)

In [0]:
a=df_to_tensor(train_x)

In [0]:
 class Loss(nn.Module):
        def __init__(self):

          super(Loss, self).__init__()
          self.mse_loss = nn.MSELoss(reduction="sum")
        def forward(self, recon_x, x, mu, logvar):
          MSE = self.mse_loss(recon_x, x)
          KLD = -0.5 * torch.sum(1 + logvar - mu**(2)-logvar.exp())
          return MSE + KLD

In [0]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [0]:
loss_=Loss()
a = a.cuda()
model =model.cuda()

In [0]:
from tqdm import tqdm_notebook as tqdm

In [0]:
def train(epoch):
    model.train()
    train_loss = 0
    for j in tqdm(range(epoch)):
      for i in tqdm(range(a.shape[0])):
          data = a[i]
          optimizer.zero_grad()
          recon_batch, mu, logvar = model(data)
          loss = loss_.forward(recon_batch, data, mu, logvar)
          loss.backward()
          train_loss += loss.item()
          optimizer.step()
          if i % 10 == 0:
              print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:}'.format(epoch, i, 111778,
                100. * i / 111778,
                loss.item() / len(data)))
       
      print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, (train_loss / len(a.shape[0]))))

In [0]:
train(1)

Streaming output truncated to the last 5000 lines.
Train Epoch: 1 [4/111778 (0%)]	Loss: 0.8758658766746521
Train Epoch: 1 [4/111778 (0%)]	Loss: 0.3535920977592468
Train Epoch: 1 [4/111778 (0%)]	Loss: 1.083134412765503
Train Epoch: 1 [4/111778 (0%)]	Loss: 0.6166454553604126
Train Epoch: 1 [4/111778 (0%)]	Loss: 0.43629878759384155
Train Epoch: 1 [4/111778 (0%)]	Loss: 0.23282954096794128
Train Epoch: 1 [4/111778 (0%)]	Loss: 0.6201561689376831
Train Epoch: 1 [4/111778 (0%)]	Loss: 0.7457283735275269
Train Epoch: 1 [4/111778 (0%)]	Loss: 0.25081369280815125
Train Epoch: 1 [4/111778 (0%)]	Loss: 0.7642190456390381
Train Epoch: 1 [4/111778 (0%)]	Loss: 0.14985507726669312
Train Epoch: 1 [4/111778 (0%)]	Loss: 0.3471640646457672
Train Epoch: 1 [4/111778 (0%)]	Loss: 0.5624222755432129
Train Epoch: 1 [4/111778 (0%)]	Loss: 0.3795356750488281
Train Epoch: 1 [4/111778 (0%)]	Loss: 1.4476513862609863
Train Epoch: 1 [4/111778 (0%)]	Loss: 0.3404408097267151
Train Epoch: 1 [4/111778 (0%)]	Loss: 0.45274400711

In [0]:
loss=Loss()
loss=loss.forward(a[0],a[0],df_to_tensor(pd.DataFrame([1,1,1,1])),df_to_tensor(pd.DataFrame([1,1,1,1])))

In [20]:
a.shape[0]

111778

In [0]:
df_to_tensor(pd.DataFrame([1,1,1,1])).exp()

tensor([[2.7183],
        [2.7183],
        [2.7183],
        [2.7183]], device='cuda:0')